<a href="https://colab.research.google.com/github/Guanhuchang/AGNet/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import time

# from model import Model

train_dataset = torchvision.datasets.CIFAR10(
    root="data",
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True)
test_dataset = torchvision.datasets.CIFAR10(
    root="data",
    train=False,
    transform=transforms.ToTensor(),
    download=True)
# length 长度
train_dataset_size = len(train_dataset)
test_dataset_size = len(test_dataset)
# 如果train_data_size=10,训练数据集的长度为：10
print("训练数据集的长度为：{}".format(train_dataset_size))
print("测试数据集的长度为：{}".format(test_dataset_size))
# 利用DataLoader来加载数据集
train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)


# 搭建神经网络
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(in_features=1024, out_features=64),
            nn.Linear(in_features=64, out_features=10)
        )

    def forward(self, x):
        x = self.model1(x)
        return x


model = Model()
if torch.cuda.is_available():
    model = model.cuda()
# 损失函数
loss_fn = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()
# 优化器
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 记录训练的轮次
epoch = 10
# 添加tensorboard
writer = SummaryWriter("logs_train")
start_time = time.time()
for i in range(epoch):
    print("-----------第{}轮训练开始----------".format(i + 1))

    # 训练步骤开始
    model.train()  #
    for data in train_dataloader:
        imgs, targets = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            targets = targets.cuda()
        outputs = model(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(end_time-start_time)
            print("训练次数：{}，loss:{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试集开始
    model.eval()
    total_test_loss = 0
    total_accurary = 0
    with torch.no_grad():  # 调用不调参
        for data in test_dataloader:
            imgs, targets = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets = targets.cuda()
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_step + loss.item()
            accurary = (outputs.argmax(1) == targets).sum()
            total_accurary = total_accurary + accurary

    print(f"整体测试集上的Loss:{total_test_loss}")
    print("整体测试集上的正确率：{}".format(total_accurary / test_dataset_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accurary / test_dataset_size, total_test_step)
    total_test_step = total_test_step + 1

    torch.save(model, "model_{}.pth".format(i))
    # torch.save(model.state_dict(),"tudui_{}.pth".format(i))
    print("模型已保存")

writer.close()





  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000
-----------第1轮训练开始----------
5.432437181472778
训练次数：100，loss:2.2822301387786865
7.030749082565308
训练次数：200，loss:2.2720367908477783
8.629156589508057
训练次数：300，loss:2.2430758476257324
10.23144268989563
训练次数：400，loss:2.1626136302948
11.812824249267578
训练次数：500，loss:2.061710834503174
13.359412670135498
训练次数：600，loss:2.0142757892608643
14.96237587928772
训练次数：700，loss:1.9950538873672485
整体测试集上的Loss:1.9612653255462646
整体测试集上的正确率：0.2897000014781952
模型已保存
-----------第2轮训练开始----------
18.89151167869568
训练次数：800，loss:1.874008297920227
20.5260591506958
训练次数：900，loss:1.8340866565704346
22.149154901504517
训练次数：1000，loss:1.875569224357605
23.821651935577393
训练次数：1100，loss:1.9387767314910889
25.47675061225891
训练次数：1200，loss:1.7062679529190063
27.087315797805786
训练次数：1300，loss:1.6181293725967407
28.732834100723267
训练次数：1400，loss:1.7102975845336914
30.399874687194824
训练次数：1500，loss:1.7456

In [ ]:
!tensorboard --logdir=log_train


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.7.0 at http://localhost:6006/ (Press CTRL+C to quit)
